In [9]:
import math
import numpy
import openmm.unit
from openff.toolkit.topology import Molecule, Topology
from openff.toolkit.typing.engines.smirnoff import ForceField, ParameterList
from openff.units import Quantity, unit
from smirnoff_plugins.utilities.openmm import simulate


In [15]:
def build_force_field() -> ForceField:    
    force_field = ForceField(load_plugins=True)
    
    improper_handler = force_field.get_parameter_handler(
        "ImproperTorsions"
    )
    
    improper_handler.add_parameter(
        {
            "smirks": "[#6:1]=[#6:2]-[#7:3]-[#6:4]",
            "length_eq": 1.2 * unit.angstrom,
            "k": 5.0 * unit.kilocalorie / unit.mole / unit.angstrom**2,
        }
    )

    return force_field


In [16]:
force_field = build_force_field()
force_field = ForceField("improper-torsion-harmonic-force-field.offxml")

molecule: Molecule = Molecule.from_smiles("C=O")
molecule.generate_conformers(n_conformers=1)

n_molecules = 100 

topology: Topology = Topology.from_molecules([molecule] * n_molecules)

topology.box_vectors = Quantity(
    numpy.eye(3) * math.ceil(n_molecules ** (1 / 3) + 2) * 2.5,
    unit.angstrom,
)

positions = openmm.unit.Quantity(
    numpy.vstack(
        [
            (
                molecule.conformers[0].m_as(unit.angstrom)
                + numpy.array([[x, y, z]]) * 2.5
            )
            for x in range(math.ceil(n_molecules ** (1 / 3)))
            for y in range(math.ceil(n_molecules ** (1 / 3)))
            for z in range(math.ceil(n_molecules ** (1 / 3)))
        ]
    ),
    openmm.unit.angstrom,
)

simulate(
    force_field=force_field,
    topology=topology,
    positions=positions,
    box_vectors=None if n_molecules == 1 else topology.box_vectors.to_openmm(),
    n_steps=2000,
    temperature=300.0,
    pressure=None if n_molecules == 1 else 1.0 * openmm.unit.atmosphere,
    platform="Reference" if n_molecules == 1 else "OpenCL",
    output_directory="simulation-output",
)



SMIRNOFFSpecError: <class 'openff.toolkit.typing.engines.smirnoff.parameters.ImproperTorsionHandler.ImproperTorsionType'> require the following missing parameters: ['periodicity', 'phase']. Defined kwargs are ['k', 'length_eq', 'smirks']